## Load Dataset

In [1]:
import pandas as pd

# Load CSV file
df = pd.read_csv('GoogleReview_data_cleaned.csv')

# Display the first few rows
df.head()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.11/site-p

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.11/site-p

AttributeError: _ARRAY_API not found

,Author,Rating,Review,Restaurant,Location
0,Jia Pin Lee,4.0,Came here for the High Tea. Great service espe...,Cuisines Restaurant,Ipoh
1,Chui Yi Lum,2.0,"5 stars for the service, even though some of t...",Cuisines Restaurant,Ipoh
2,liezel wong,1.0,"Hi, thank you for your service. But! i feel so...",Cuisines Restaurant,Ipoh
3,Nazri Nor,1.0,I have the worse buffer dinner ever so far. Th...,Cuisines Restaurant,Ipoh
4,Fakru Imran's Channel,5.0,"That's are Known 5 Elmark "" 9H72 "" & KDK "" 3 K...",Cuisines Restaurant,Ipoh


## Preprocessing Setup

In [2]:
import nltk

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')


/opt/anaconda3/lib/python3.11/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 2.0.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## Preprocessing Function (with stemming)

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_review(text):
    text = text.lower()
    
    # Tokenize without using sentence tokenizer (this avoids 'punkt_tab')
    tokens = word_tokenize(text, preserve_line=True)
    
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    tokens = [stemmer.stem(word) for word in tokens]
    
    return tokens



In [ ]:
df['Processed_Tokens'] = df['Review'].apply(clean_review)
df[['Review', 'Processed_Tokens']].head()


## Part-of-Speech (POS) Tagging

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')

# Tag POS for the first review
nltk.pos_tag(df['Processed_Tokens'][0])


## Clean Tokens Without Stemming

In [ ]:
# to keep original word forms for better accuracy in POS tagging and NER

def clean_review_no_stem(text):
    text = text.lower()
    tokens = word_tokenize(text, preserve_line=True)
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens


In [ ]:
df['Clean_Tokens'] = df['Review'].apply(clean_review_no_stem)
nltk.pos_tag(df['Clean_Tokens'][0])


## Part-of-Speech (POS) Tagging

In [ ]:
# assign grammatical roles (noun, verb, adjective, etc.) to each token using `nltk.pos_tag`

# POS tagging all reviews

df['POS_Tags'] = df['Clean_Tokens'].apply(nltk.pos_tag)
df[['Review', 'POS_Tags']].head()


## Word Cloud

In [ ]:
!pip install wordcloud


In [ ]:
# this visualization shows the most frequent words used in the Google Reviews

from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Join all cleaned tokens into one string
all_words = ' '.join([' '.join(tokens) for tokens in df['Clean_Tokens']])

# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_words)

# Show it
plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Most Frequent Words in Google Reviews", fontsize=18)
plt.show()


## Named Entity Recognition (NER)

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


In [ ]:
# we use SpaCy to detect real-world entities like locations, organizations, and people mentioned in the reviews

import spacy

# Load SpaCy English model
nlp = spacy.load("en_core_web_sm")

# Choose a review and process it
text = df['Review'][0]
doc = nlp(text)

# Extract named entities (NER for one review)
for ent in doc.ents:
    print(f"{ent.text} → {ent.label_}")


In [ ]:
# Store NER for All Reviews
df['Named_Entities'] = df['Review'].apply(lambda text: [(ent.text, ent.label_) for ent in nlp(text).ents])
df[['Review', 'Named_Entities']].head()

## Summary of Completed Tasks

**Basic NLP Preprocessing:**
- Tokenization - word_tokenize()
- Case Folding - text.lower()   
- Punctuation Removal - word.isalpha()
- Stopword Removal - word not in stop_words
- Stemming  - stemmer.stem(word)
- POS Tagging

**Text Processing Tasks:**
- Named Entity Recognition (SpaCy)
- Word Cloud